In [ ]:
import ultralytics
from ultralytics import YOLO
import math
import cv2
import numpy as np
from flask import Flask, render_template, Response, request
import openvino as ov

In [ ]:
# model
model = YOLO('C:\\Users\\SIMATIC\\Documents\\helmet_n_openvino_model', task='detect') #C:\Users\SIMATIC\Documents\helmet_n.pt

# object classes
classNames = ["cabeca","capacete","pessoa"]


In [ ]:
#web server

app = Flask(__name__)

#webcam

camera = cv2.VideoCapture(0)
#camera.set(3, 640)
#camera.set(4, 640)

def gen_frames():
    while True:
        success, frame = camera.read()
        results = model(frame, stream=True)
        if not success:
            break
        else:
            #Faz as caixas de identificaçao
            for r in results:
                boxes = r.boxes
                for box in boxes:
                    # bounding box
                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

                    # put box in cam
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 255), 3)

                    # confidence
                    confidence = math.ceil((box.conf[0]*100))/100
                    print("Confidence --->",confidence)

                    # class name
                    cls = int(box.cls[0])
                    print("Class name -->", classNames[cls])

                    # object details
                    org = [x1, y1]
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    fontScale = 1
                    color = (255, 0, 0)
                    thickness = 2

                    cv2.putText(frame, classNames[cls] + " " + str(confidence), org, font, fontScale, color, thickness)

            #Projeta imagem no servidor web
            ret, buffer = cv2.imencode('.jpg', frame)
            frame = buffer.tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route('/')
def index():
    return Response(gen_frames(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

app.run(host='0.0.0.0', port=8020)